In [1]:
from tensorflow.contrib.data import Dataset, Iterator
import pandas as pd
import numpy as np
import os
from daewoo_module import *
from tqdm import tqdm_notebook as tqdm

root_dir = ".\\input_data"
# img_dir = os.path.join(root_dir, '/figure'). 이미지는 os.path.join쓰면 안되는경우가 있음
img_dir = "./input_data/figure/"
logs_path = os.path.join(root_dir, "graph")

label = pd.read_csv(os.path.join(root_dir, 'description.csv'), engine='python')
label = label['WVHT ft.y'].values
label = ((label - np.mean(label))/np.std(label)).reshape(-1,1)

batch_size = 16
epochs = 2

C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
img = np.array([img_dir + x for x in os.listdir(img_dir)])

np.random.seed(1234)
idx = np.random.permutation(len(img))
tr_idx = idx[:round(0.8*len(idx))]
ts_idx = idx[round(0.8*len(idx)):]

train_img = img[tr_idx]
train_label = label[tr_idx]
test_img = img[ts_idx]
test_label = label[ts_idx]

train_img_tensor = tf.constant(train_img)
train_label_tensor = tf.constant(train_label)
test_img_tensor = tf.constant(test_img)
test_label_tensor = tf.constant(test_label)

train_imgs = Dataset.from_tensor_slices((train_img_tensor,train_label_tensor))
train_imgs = train_imgs.map(input_tensor).shuffle(buffer_size=1000).batch(batch_size)
test_imgs = Dataset.from_tensor_slices((test_img_tensor,test_label_tensor))
test_imgs = test_imgs.map(input_tensor).shuffle(buffer_size=1000).batch(batch_size)

train_iterator = train_imgs.make_initializable_iterator()
test_iterator = test_imgs.make_initializable_iterator()

train_x, train_y = train_iterator.get_next()
test_x, test_y = test_iterator.get_next()

train_batches = len(train_img) // batch_size
test_batches = len(test_img) // batch_size

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [5]:
model_name = "VGG16_"
model = VGG16(bn=True)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))
    
    for i in tqdm(range(epochs)):
        print("Training!")
        sess.run(train_iterator.initializer)
        sess.run(test_iterator.initializer)
        for j in tqdm(range(train_batches)):
            summary, _, loss_ = sess.run([model.merged_summary_op, model.train, model.loss],
                                        feed_dict={model.x:train_x,
                                                   model.y:train_y,
                                                   model.learning_rate:0.001})
            if j % 10 == 0:
                print("Training Epoch : {}, Iter : {}, Loss : {:.4f}".format(i+1, j, loss_))
                train_summary_writer.add_summary(summary, j)
                summary, loss_ = sess.run([model.merged_summary_op, model.loss],
                                         feed_dict={model.x:test_x, model.y:test_y})
                print("Validation Iter : {}, Loss : {:.4f}".format(j, loss_))
                test_summary_writer.add_summary(summary, j)
    
    print("-----------End of training-------------")
            
            

In [ ]:
tf.reset_default_graph()
sess.close()
